In [252]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics import accuracy_score 
import pickle

In [193]:
train=pd.read_csv("train.csv")

In [194]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [195]:
train.drop(["Name","Ticket","Cabin","PassengerId"],axis=1,inplace=True)

In [196]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [197]:
x=train.drop(["Survived"],axis=1)
y=train["Survived"]

In [198]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [199]:
#Imputation Transformer

trf1=ColumnTransformer([
    ("Imputer_age",SimpleImputer(),[2]),
    ("Imputer_Embarked",SimpleImputer(strategy="most_frequent"),[6])
],remainder="passthrough")

In [200]:
# One Hot Encoding

trf2=ColumnTransformer([
    ("ohe_sex_embarked",OneHotEncoder(sparse_output=False,handle_unknown="ignore"),[1,6])
],remainder="passthrough")

In [201]:
#Scaling

trf3=ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10))
])


In [202]:
#Feature Selection

trf4=SelectKBest(score_func=chi2,k=8)

In [203]:
#train the model

trf5=DecisionTreeClassifier()

# Create pipeline

In [205]:
pipe=Pipeline([
    ("trf1",trf1),
    ("trf2",trf2),
    ("trf3",trf3),
    ("trf4",trf4),
    ("trf5",trf5),  
])


                                                Pipeline vs make_Pipeline

                                Pipeline requires naming of steps ,make_pipeline does not 
                             (same applies to column_transformer and make_column_transformer)
            It is recommended to use make_pipeline because it is easier to use than the pipeline one as shown below 

In [208]:
# pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [209]:
#train

pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Imputer_age',
                                                  SimpleImputer(), [2]),
                                                 ('Imputer_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8, score_func=<function chi2 at 0x1593494e0>)),
                ('trf5', DecisionTreeClassifier())])

# Explore the pipeline


In [237]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Imputer_age', SimpleImputer(), [2]),
                                 ('Imputer_Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x1593494e0>),
 'trf5': DecisionTreeClassifier()}

In [241]:
y_pred=pipe.predict(x_test)

In [249]:
accuracy_score(y_test,y_pred)

0.6256983240223464

# Exporting the pipeline to use for production code

In [254]:
pickle.dump(pipe,open("models/pipe.pkl","wb"))